In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm
from random import randint
from time import sleep




In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&datatype=csv&outputsize=compact&apikey=9YUPW0WRVEKUFF9L'
df_stock = pd.read_csv(url)
df_stock = df_stock.drop(columns=['volume'])
df_stock.to_csv('stock_data.csv', index=False)


In [136]:
driver = webdriver.Chrome()


In [137]:
url = "https://trendspider.com/markets/congress-trading/"
driver.get(url)
time.sleep(5)


In [138]:
headers = driver.find_elements(By.CSS_SELECTOR, "#main-content table thead th")
header_names = [header.text.strip() for header in headers]


In [139]:
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text.strip().replace('\n', ' ') for col in cols])

In [140]:
for i in tqdm(range(2, 811)):
    check = f'[aria-label="Current Page, Page {i}."]'
    site = f'[aria-label="Go to page {i}."]'
    while driver.find_elements(By.CSS_SELECTOR, check) == []:
        try:
            driver.find_elements(By.CSS_SELECTOR, site)[0].click()
        except Exception as e:
            time.sleep(randint(10,20)/10)
    

    sleep(randint(10,20)/10)
    rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")
    
    sleep(randint(10,20)/10)
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text.strip().replace('\n', ' ') for col in cols])

100%|██████████| 809/809 [1:58:56<00:00,  8.82s/it]  


In [141]:
df = pd.DataFrame(data, columns=header_names)
df[['Type', 'Amount']] = df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
df = df.drop(columns=['Transaction', 'Excess return *'])


In [181]:
#df.to_csv('senators_trading.csv', index=False)
driver.quit()


In [21]:
df = pd.read_csv('senators_trading.csv')

UPDATE

In [177]:
data_trading = pd.read_csv("senators_trading.csv")

In [182]:
data_trading = data_trading.drop(index=data_trading.index[:100])


In [183]:
driver = webdriver.Chrome()

In [184]:
url = "https://trendspider.com/markets/congress-trading/"
driver.get(url)
time.sleep(5)


In [185]:
headers = driver.find_elements(By.CSS_SELECTOR, "#main-content table thead th")
header_names = [header.text.strip() for header in headers]

In [186]:
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text.strip().replace('\n', ' ') for col in cols])

    


df = pd.DataFrame(data, columns=header_names)
df[['Type', 'Amount']] = df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
df = df.drop(columns=['Transaction', 'Excess return *'])

last_note = df.iloc[-1]

In [190]:
i = 2
while ((data_trading == last_note).all(axis=1)).any() == False:
    data = []
    check = f'[aria-label="Current Page, Page {i}."]'
    site = f'[aria-label="Go to page {i}."]'
    while driver.find_elements(By.CSS_SELECTOR, check) == []:
        try:
            driver.find_elements(By.CSS_SELECTOR, site)[0].click()
        except Exception as e:
            time.sleep(randint(10,20)/10)
    sleep(randint(10,50)/10)
    rows = driver.find_elements(By.CSS_SELECTOR, "#main-content table tbody tr")
    
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text.strip().replace('\n', ' ') for col in cols])

    help_df = pd.DataFrame(data, columns=header_names)
    help_df[['Type', 'Amount']] = help_df['Transaction'].str.extract(r'(\w+)\s\$(.*)')
    help_df = help_df.drop(columns=['Transaction', 'Excess return *'])

    df = pd.concat([df, help_df])


    i += 1
    last_note = df.iloc[-1]

    print(last_note)

driver.quit()

mask = ~df.apply(tuple, axis=1).isin(data_trading.apply(tuple, axis=1))
result = df[mask]

data_trading = pd.concat([result, data_trading])